In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')
df.drop(columns=['ID'], inplace=True)

print(df.head())

    timestamp item corporation location  supply(kg)  price(원/kg)
0  2019-01-01   TG           A        J         0.0          0.0
1  2019-01-02   TG           A        J         0.0          0.0
2  2019-01-03   TG           A        J     60601.0       1728.0
3  2019-01-04   TG           A        J     25000.0       1408.0
4  2019-01-05   TG           A        J     32352.0       1250.0


In [4]:
# 아이템별로 데이터프레임 나누기

items = df['item'].unique()
dfs_list = [] # 아이템별 데이터프레임 저장된 리스트

for item in items:
    item_df = df[df['item'] == item]
    globals()[f'df_{item}'] = item_df
    dfs_list.append(item_df)

print(dfs_list)
print(items)
# items 순서대로 dfs_list가 저장되어 있음

[        timestamp item corporation location  supply(kg)  price(원/kg)
0      2019-01-01   TG           A        J         0.0          0.0
1      2019-01-02   TG           A        J         0.0          0.0
2      2019-01-03   TG           A        J     60601.0       1728.0
3      2019-01-04   TG           A        J     25000.0       1408.0
4      2019-01-05   TG           A        J     32352.0       1250.0
...           ...  ...         ...      ...         ...          ...
15225  2023-02-27   TG           E        S     24204.0       3418.0
15226  2023-02-28   TG           E        S     13587.0       3141.0
15227  2023-03-01   TG           E        S     16187.0       4235.0
15228  2023-03-02   TG           E        S     17830.0       3960.0
15229  2023-03-03   TG           E        S     26157.0       3791.0

[15230 rows x 6 columns],         timestamp item corporation location  supply(kg)  price(원/kg)
15230  2019-01-01   CR           A        J         0.0          0.0
15231 

In [5]:
# whether은 zero-one list로서, i번째 전처리를 실행할지 아닌지를 결정함

avg_month_list = [] # 과일별로 함수 돌리게 되면 ['TG' 'CR' 'CB' 'RD' 'BC'] 순서대로 저장됨
avg_year_list = [] 
avg_location_list = [] 
avg_corporation_list = []

def preprocessing_for_train(df, whether) :
    
    df['year'] = pd.to_datetime(df['timestamp']).dt.year
    df['month'] = pd.to_datetime(df['timestamp']).dt.month
    df['day'] = pd.to_datetime(df['timestamp']).dt.day
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # 첫 번째 고려할 수 있는 칼럼 : month별 평균값
    if whether[0] == 1 :
        avg = []
        month_values = df['month'].unique().tolist()
        for i in month_values :
            df.loc[df['month'] == i, 'avg_month'] = df.loc[df['month'] == i, 'price(원/kg)'].mean()
            avg_month = df.loc[df['month'] == i, 'price(원/kg)'].mean()
            avg.append([i, avg_month])
        avg_month_list.append(avg)
        
    # 두 번째 고려할 수 있는 칼럼 : year별 평균값
    if whether[1] == 1 :
        avg = []
        year_values = df['year'].unique().tolist()
        for i in year_values :
            df.loc[df['year'] == i, 'avg_year'] = df.loc[df['year'] == i, 'price(원/kg)'].mean()
            avg_year = df.loc[df['year'] == i, 'price(원/kg)'].mean()
            avg.append([i, avg_year])
        avg_year_list.append(avg)       
        
    # 세 번째 고려할 수 있는 칼럼 : location별 평균값
    if whether[2] == 1 :
        avg = []
        location_values = df['location'].unique().tolist()
        for i in location_values :
            df.loc[df['location'] == i, 'avg_location'] = df.loc[df['location'] == i, 'price(원/kg)'].mean()
            avg_location = df.loc[df['location'] == i, 'price(원/kg)'].mean()
            avg.append([i, avg_location])
        avg_location_list.append(avg)   
    
    # 네 번째 고려할 수 있는 칼럼 : corporation별 평균값
    if whether[3] == 1 :
        avg = []
        corporation_values = df['corporation'].unique().tolist()
        for i in corporation_values :
            df.loc[df['corporation'] == i, 'avg_corporation'] = df.loc[df['corporation'] == i, 'price(원/kg)'].mean()
            avg_corporation = df.loc[df['corporation'] == i, 'price(원/kg)'].mean()
            avg.append([i, avg_corporation])
        avg_corporation_list.append(avg)
        
    # 다섯 번째 고려할 수 있는 칼럼 : 요일 고려해보기 (범주형 처리했음)
    if whether[4] == 1 :
        df['day_of_week'] = df['timestamp'].dt.day_name()
        df = pd.get_dummies(df, columns=['day_of_week'])
    
    # 여섯 번째 고려할 수 있는 칼럼 : 1년 단위로 삼각함수 변환
    if whether[5] == 1 :
        df['sin_timestamp'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        df['cos_timestamp'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofyear / 365)
        
    if whether[6] == 1 :
        df.drop(columns=['month'], inplace=True)
        
    if whether[7] == 1 :
        df.drop(columns=['day'], inplace=True)
        
    if whether[8] == 1 :
        df.drop(columns=['year'], inplace=True)       
    
    if whether[9] == 1:
        df = pd.get_dummies(df, columns=['corporation'])
    else:
        df.drop(columns=['corporation'], inplace=True)
    
    if whether[10] == 1 :
        df = pd.get_dummies(df, columns=['location']) 
    else :  
        df.drop(columns=['location'], inplace=True)  
              
    df.drop(columns=['supply(kg)'], inplace=True)    
    # df.drop(columns=['timestamp'], inplace=True)
    # df.drop(columns=['item'], inplace=True)

        
    return df

In [6]:
from sklearn.metrics import mean_squared_error
import math
from lightgbm import LGBMRegressor

def modeling (df) :
    
    march_2019_data_indices = df[(df['timestamp'] >= '2019-03-01') & (df['timestamp'] < '2019-04-01')].index.tolist()
    
    col_list = df.columns
    col_list = [col for col in col_list if col != 'price(원/kg)' and col != 'timestamp']
    
    X = df[col_list]
    Y = df['price(원/kg)']
    
    X_train = df.loc[~df.index.isin(march_2019_data_indices), col_list]
    Y_train = df.loc[~df.index.isin(march_2019_data_indices), 'price(원/kg)']

    X_test = df.loc[df.index.isin(march_2019_data_indices), col_list]
    Y_test = df.loc[df.index.isin(march_2019_data_indices), 'price(원/kg)']
    
    model_lgbm = LGBMRegressor()
    model_lgbm.fit(X_train, Y_train)
    predictions_lgbm = model_lgbm.predict(X_test)
    mse_lgbm = mean_squared_error(Y_test, predictions_lgbm)
    print("Root Mean Squared Error: ", math.sqrt(mse_lgbm))
    
    return mse_lgbm
        

In [7]:
import itertools
binary_combinations = list(itertools.product([0, 1], repeat=11))

In [8]:
print(df)

        timestamp item corporation location  supply(kg)  price(원/kg)
0      2019-01-01   TG           A        J         0.0          0.0
1      2019-01-02   TG           A        J         0.0          0.0
2      2019-01-03   TG           A        J     60601.0       1728.0
3      2019-01-04   TG           A        J     25000.0       1408.0
4      2019-01-05   TG           A        J     32352.0       1250.0
...           ...  ...         ...      ...         ...          ...
59392  2023-02-27   RD           F        J    452440.0        468.0
59393  2023-02-28   RD           F        J    421980.0        531.0
59394  2023-03-01   RD           F        J    382980.0        574.0
59395  2023-03-02   RD           F        J    477220.0        523.0
59396  2023-03-03   RD           F        J    427520.0        529.0

[59397 rows x 6 columns]


In [327]:
min = 100000
whether_min = [0] * 11
for whether in binary_combinations :
    df1 = df.copy() 
    total_df = preprocessing_for_train(df1, whether)
    total_df = pd.get_dummies(total_df, columns=['item'])
    if math.sqrt(modeling(total_df)) < min :
        min = math.sqrt(modeling(total_df))
        whether_min = whether
print(min)
print(whether_min)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 61
[LightGBM] [Info] Number of data points in the train set: 58188, number of used features: 8
[LightGBM] [Info] Start training from score 1126.812143
Root Mean Squared Error:  1312.1987329683036
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000368 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 61
[LightGBM] [Info] Number of data points in the train set: 58188, number of used features: 8
[LightGBM] [Info] Start training from score 1126.812143
Root Mean Squared Error:  1312.1987329683036
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.0

In [10]:
total = 0
for df_item in dfs_list :
    min_rmse = 100000
    min_mse = 0
    whether_min = [0] * 11
    for whether in binary_combinations :
        df1 = df_item.copy() 
        total_df = preprocessing_for_train(df1, whether)
        total_df.drop(columns=['item'], inplace=True)
        if math.sqrt(modeling(total_df)) < min :
            min_mse = modeling(total_df)
            min_rmse = math.sqrt(min_mse)
            whether_min = whether
    total = min_mse + total
    print('mse: ', min_mse)
    print('whether_min: ', whether_min)
total_rmse = math.sqt(total)
        

15230
10661
7615
12184
13707
